# Reconhecimento de dígitos do painel de senha do restaurante do Insper

## Descrição do projeto e problema a ser estudado

O objetivo deste projeto é desenvolver uma solução para permitir que os clientes do restaurante do 6° andar do prédio 2 da faculdade Insper possam acompanhar a senha atual sem precisar estar próximo do painel. Notamos no nosso dia a dia que muitos alunos não conseguem ouvir ou ver a senha atual, já que a maior parte dos assentos fica longe do painel.

Para resolver esse problema, utilizaremos técnicas de reconhecimento de dígitos por meio de imagens. Inicialmente, faremos o reconhecimento de dígitos do painel de senha com o uso de OpenCV e OCR em um Jupyter Notebook. Com isso, conseguiremos extrair as informações necessárias e armazená-las em um banco de dados. Posteriormente, poderemos criar um aplicativo que exibe a senha atual, permitindo que os clientes possam acompanhar o seu pedido sem precisar estar próximo do painel.

## Origem dos dados

Os dados serão gerados por meio de um vídeo feito por nós do painel de senha do restaurante do Insper. Utilizaremos esse vídeo para treinar o modelo de reconhecimento de dígitos.

## Técnicas para extrair e tratar os dados

Os dados serão extraídos a partir de um vídeo que captura o painel de senha do restaurante do Insper em funcionamento. Faremos o tratamento das imagens por meio de técnicas de OpenCV e OCR para extrair os dígitos do painel. 

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import easyocr


C:\Users\tutuc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\detection\anchor_utils.py:63: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  device: torch.device = torch.device("cpu"),


In [2]:
def digit_recognition(img):
    # Carrega a imagem
    # img = cv2.imread(data_dir + "IMG_4124.jpg")
    # scale image to fit 'window'
    img = cv2.resize(img, (0, 0), fx=0.3, fy=0.3)
    # Convert to grayscale

    WIDTH = img.shape[1]
    HEIGHT = img.shape[0]

    img_cropped = img[int(HEIGHT / 2 - 200):int(HEIGHT / 2 + 200), int(WIDTH / 2 - 200):int(WIDTH / 2 + 200)]
    img_gray_cropped = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2GRAY)
    # Apply thresholding
    _, img_thresh = cv2.threshold(img_gray_cropped, 250, 255, cv2.THRESH_BINARY)

    # Crop center square

    mask = img_thresh.copy()
    mask = cv2.blur(mask, (5, 5))

    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((10, 10), np.uint8))

    # INVERT COLOR
    mask = cv2.bitwise_not(mask)

    reader = easyocr.Reader(lang_list=['pt'], gpu=False)
    texto = reader.readtext(mask, detail=0)

    texto = texto[0].replace(" ", "")

    img_rgb = img_cropped.copy()
    cropped_height = img_rgb.shape[0]
    cropped_width = img_rgb.shape[1]

    cv2.putText(img_rgb, texto, (int(cropped_width / 2), int(cropped_height / 2 + 50)), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (255, 0, 0), 2)

    return img_rgb

In [6]:
# Define the directory where the images are stored
image_dir = "data/img/"

# Loop through all the image files in the directory
for filename in os.listdir(image_dir):
    
    # Load the image
    img = cv2.imread(os.path.join(image_dir, filename))
    
    # Call the digit_recognition function
    img = digit_recognition(img)
    
    # Display the image using Matplotlib
    plt.imshow(img)
    plt.title(filename)
    plt.show()



Using CPU. Note: This module is much faster with a GPU.


RuntimeError: Numpy is not available

In [12]:
cap = cv2.VideoCapture("data/video/IMG_4123.MOV")

while True:
    ret, frame = cap.read()
    if ret == False:
        break
    frame = digit_recognition(frame)
    cv2.imshow("Video", frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
